In [ ]:
%pylab inline
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms
from torchvision.datasets import MNIST

In [ ]:
train = MNIST('./data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

test = MNIST('./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )
batch_size = 64

dataloader_args = dict(shuffle=True, batch_size=batch_size)
train_loader = dataloader.DataLoader(train, **dataloader_args)
test_loader = dataloader.DataLoader(test, **dataloader_args)

In [ ]:
train_data = train.train_data
train_data = train.transform(train_data.numpy())

print('[Train]')
print(' - Numpy Shape:', train.train_data.cpu().numpy().shape)
print(' - Tensor Shape:', train.train_data.size())
print(' - Transformed Shape:', train_data.size())
print(' - min:', torch.min(train_data))
print(' - max:', torch.max(train_data))
print(' - mean:', torch.mean(train_data))
print(' - std:', torch.std(train_data))
print(' - var:', torch.var(train_data))

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.fc1 = nn.Linear(784, 392)
        
        self.fc2 = nn.Linear(392, 50)
        
        self.fc3 = nn.Linear(50, 10)
        
        
    def forward(self, x):
        x = x.view((-1, 784))
        h = self.fc1(x)
        h = F.relu(h)
        
        h = self.fc2(h)
        h = F.relu(h)
        
        h = self.fc3(h)
        out = h
        return out

model = Model()
print(model)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train_loss, train_acc, test_acc = [], [], []
for epoch in range(100):
    avg_loss, avg_acc, total = 0.0, 0.0, 0.0
    
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        one_hot = torch.eye(10).index(target) * 10
        data, target = Variable(data), Variable(target)
        one_hot = Variable(one_hot)
        optimizer.zero_grad()
        y_pred = model(data) 
        loss = nn.MSELoss()(y_pred, one_hot)
        avg_loss += loss.data[0] * int(data.size()[0])
        loss.backward()
        optimizer.step()
        prediction = y_pred.data.max(1)[1]
        avg_acc += prediction.eq(target.data).sum()
        total += int(data.size()[0])
    
    avg_loss = avg_loss / total
    avg_acc = avg_acc / total * 100
    train_loss.append(avg_loss)
    train_acc.append(avg_acc)
    print("Train Epoch {}\tLoss: {:.6f}\tAccuracy: {:.6f}".format(epoch, avg_loss, avg_acc))
    
    evaluate_x = Variable(test_loader.dataset.test_data.type_as(torch.FloatTensor()))
    evaluate_y = Variable(test_loader.dataset.test_labels)
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    avg_test_acc = pred.eq(evaluate_y.data).sum() / float(evaluate_y.size()[0]) * 100
    test_acc.append(avg_test_acc)
    print("Test Epoch {}\tAccuracy: {:.6f}".format(epoch, avg_test_acc))

In [ ]:
import csv
with open("linear.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(zip(train_loss, train_acc, test_acc))